In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Bird Flocking Behaviour

In [ ]:
class Bird():
    def __init__(self, x, y, vx, vy, dt, max_speed, separation_radius, cohesion_radius, alignment_radius):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.dt = dt
        self.max_speed = max_speed
        self.separation_radius = separation_radius
        self.cohesion_radius = cohesion_radius
        self.alignment_radius = alignment_radius
        
    # Avoid hitting neighbours by moving away from them
    def separation(self, birds):
        separation = [0, 0]
        for bird in birds:
            # If the neighbour is within the separation radius
            if bird != self and (bird.x - self.x)**2 + (bird.y - self.y)**2 < self.separation_radius**2:
                separation[0] += (self.x - bird.x) / self.dt
                separation[1] += (self.y - bird.y) / self.dt
        return separation
    
    # Align velocity with the average velocity of nearby neighbours
    def alignment(self, birds):
        alignment = [0, 0]
        count = 0
        for bird in birds:
            # If the neighbour is within the alignment radius
            if bird != self and (bird.x - self.x)**2 + (bird.y - self.y)**2 < self.alignment_radius**2:
                alignment[0] += bird.vx
                alignment[1] += bird.vy
                count += 1
                
        # Calculate the average alignment
        if count > 0:
            alignment[0] /= count
            alignment[1] /= count
        return alignment
    
    # Move towards the average position of nearby neighbours
    def cohesion(self, birds):
        cohesion = [0, 0]
        count = 0
        for bird in birds:
            # If the neighbour is within the cohesion radius
            if bird != self and (bird.x - self.x)**2 + (bird.y - self.y)**2 < self.cohesion_radius**2:
                cohesion[0] += bird.x
                cohesion[1] += bird.y
                count += 1
                
        # Calculate the average cohesion
        if count > 0:
            cohesion[0] /= count
            cohesion[1] /= count
            cohesion[0] = (cohesion[0] - self.x) / self.dt
            cohesion[1] = (cohesion[1] - self.y) / self.dt
        return cohesion
        
    # Find the new velocity and position
    def move(self, birds):
        # Get the separation, cohesion, and alignment velocity vectors
        separation = self.separation(birds)
        cohesion = self.cohesion(birds)
        alignment = self.alignment(birds)
        self.vx += separation[0] + cohesion[0] + alignment[0]
        self.vy += separation[1] + cohesion[1] + alignment[1]
        
        # Limit the speed
        speed = np.sqrt(self.vx**2 + self.vy**2)
        if speed > self.max_speed:
            self.vx = self.vx * self.max_speed / speed
            self.vy = self.vy * self.max_speed / speed
        self.x += self.vx * self.dt
        self.y += self.vy * self.dt

In [ ]:
class Flock():
    def __init__(self, bird_count, map_width, map_height, separation_radius, cohesion_radius, alignment_radius, dt, max_speed):
        self.bird_count = bird_count
        self.map_width = map_width
        self.map_height = map_height
        self.separation_radius = separation_radius
        self.cohesion_radius = cohesion_radius
        self.alignment_radius = alignment_radius
        self.dt = dt
        self.birds = [Bird(np.random.uniform(0, map_width), 
                           np.random.uniform(0, map_height), 
                           np.random.uniform(-1, 1), 
                           np.random.uniform(-1, 1), 
                           dt, 
                           max_speed,
                           separation_radius, 
                           cohesion_radius, 
                           alignment_radius) for _ in range(bird_count)]
            
    # Update the position and velocity of all birds
    def move(self):
        for bird in self.birds:
            bird.move(self.birds)
            
            # Use a periodic boundary condition which wraps around the map
            bird.x = bird.x % self.map_width
            bird.y = bird.y % self.map_height
        
        # Get the positions of all birds for plotting
        positions = [(bird.x, bird.y) for bird in self.birds]
        velocities = [(bird.vx, bird.vy) for bird in self.birds]
        return positions, velocities

In [ ]:
# Run the simulation and get results for all iterations
def simulate_flock(bird_count=50, map_width=100, map_height=100, separation_radius=10, cohesion_radius=15, alignment_radius=15, dt=0.001, max_speed=1, iterations=100):
    flock = Flock(bird_count, map_width, map_height, separation_radius, cohesion_radius, alignment_radius, dt, max_speed)
    all_positions = []
    all_velocities = []
    for _ in range(iterations):
        positions, velocities = flock.move()
        all_positions.append(positions)
        all_velocities.append(velocities)
    
    def plot_iteration(i=0):
        plt.figure(figsize=(10, 10))
        x_positions = [pos[0] for pos in all_positions[i]]
        y_positions = [pos[1] for pos in all_positions[i]]
        x_velocities = [vel[0] for vel in all_velocities[i]]
        y_velocities = [vel[1] for vel in all_velocities[i]]
        
        plt.scatter(x_positions, y_positions, color='blue')
        plt.quiver(x_positions, y_positions, x_velocities, y_velocities, color='red')
        plt.xlim(0, map_width)
        plt.ylim(0, map_height)
        plt.title(f"Step {i}")
        plt.show()

    slider = widgets.IntSlider(min=0, max=len(all_positions)-1, step=1, value=0)
    widgets.interact(plot_iteration, i=slider)

In [ ]:
# simulate_flock(bird_count=50, map_width=100, map_height=100, separation_radius=8, dt=0.1, max_speed=10, iterations=200)